In [2]:
##impor every thing that are need in this project 
import os 
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import json
import pandas as pd
import traceback 
import PyPDF2


/home/sahil/Question_Answ/question/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## in this project i use gemini 

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key="")

In [4]:
## that are the format of response 

RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [5]:
## this template 1 
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

""" 
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [6]:
## create the prompt 

quiz_generation_prompt=PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
) 

## now create a chain for prompt one 

llm_chain1=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz")

/home/sahil/Question_Answ/question/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [7]:
## create prompt for second template 

quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
) 

review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review") 

In [8]:
## now use squentail chain 

generate_evaluate_chain=SequentialChain(chains=[llm_chain1, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"])

In [9]:
## now test our model 
file_path="/home/sahil/Question_Answ/data.txt"
with open(file_path, 'r') as file:
    TEXT = file.read()

In [10]:
NUMBER=10
SUBJECT="Mechin learning"
TONE="simple"

In [11]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

/home/sahil/Question_Answ/question/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [12]:
quiz=response.get("quiz")

In [13]:
quiz=quiz.split("### RESPONSE_JSON\n")[1]
quiz

'{"1": {"mcq": "Who coined the term \'machine learning\'?", "options": {"a": "Alan Turing", "b": "Arthur Samuel", "c": "Donald Hebb", "d": "Tom M. Mitchell"}, "correct": "b"}, "2": {"mcq": "What was the earliest machine learning model?", "options": {"a": "A program that played checkers", "b": "Cybertron, a learning machine", "c": "A neural network model", "d": "A decision tree"}, "correct": "a"}, "3": {"mcq": "What is the primary difference between machine learning and data mining?", "options": {"a": "Machine learning focuses on prediction, while data mining focuses on discovery.", "b": "Machine learning uses supervised learning, while data mining uses unsupervised learning.", "c": "Machine learning is used for classification, while data mining is used for regression.", "d": "Machine learning is a subset of data mining."}, "correct": "a"}, "4": {"mcq": "What is the goal of generalization in machine learning?", "options": {"a": "To improve the accuracy of the model on unseen data.", "b"

In [17]:
quiz=json.loads(quiz)

In [18]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [19]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Alan Turing | b: Arthur Samuel | c: Donald Hebb | d: Tom M. Mitchell',
  'Correct': 'b'},
 {'MCQ': 'What was the earliest machine learning model used for?',
  'Choices': 'a: Playing chess | b: Analyzing sonar signals | c: Predicting stock prices | d: Classifying cancerous moles',
  'Correct': 'a'},
 {'MCQ': 'What book introduced a theoretical neural structure based on interactions among nerve cells?',
  'Choices': 'a: Learning Machines | b: The Organization of Behavior | c: Computing Machinery and Intelligence | d: Data Compression',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is NOT a type of machine learning approach?',
  'Choices': 'a: Supervised learning | b: Unsupervised learning | c: Reinforcement learning | d: Deep learning',
  'Correct': 'd'},
 {'MCQ': 'What is the main goal of supervised learning?',
  'Choices': 'a: Finding patterns in unlabeled data | b: Learning a function to map inputs to outputs |

In [20]:
quiz=pd.DataFrame(quiz_table_data)

In [21]:
quiz.to_csv("ml.csv",index=False)